# Daniel Posada

## Interview
- Link: https://docs.google.com/document/d/1AkjuE6y4LRQqV68Pn2cN_EHCm4xm4DJU/edit
- Audio: https://drive.google.com/file/d/16VGertRVnvTST8ynfWswq8PlgwbDbMjf/view?usp=drive_link

He’s been living by Harborside Park for a year or two. The drones make him feel he’s constantly being watched even though he hasn’t done anything wrong. The hot team breaks up the encampment once a week. They toss out most of his stuff, meaning every week he needs to start his life over again. He has nowhere else to go and no money to seriously start over. 

“It’s like (being) boxed in.” 

About a month and a half ago, he got into a verbal argument with his girlfriend and shortly thereafter a drone showed up and started following him. “I feel like a target.” A police car showed up a few minutes after that. The officer stopped and searched him. It happened near the Circle K on Broadway. He didn’t understand why the police would send a drone to investigate a non-physical fight. 

He said he’s not on probation. No 4th amendment waiver in place. 

He would prefer the money spent on drones go towards housing, clothing and food. He’s from Chula Vista. 



## Load Data

In [2]:
import os

import fiona
import geopandas as gpd
import gpxpy
import numpy as np
import pandas as pd
from multiprocess.pool import Pool
from tqdm.notebook import tqdm, trange
import datetime
import geopy.distance

from loguru import logger 

fiona.drvsupport.supported_drivers['kml'] = 'rw'
fiona.drvsupport.supported_drivers['KML'] = 'rw'
storage = "/Volumes/easystore/Drones"
coi=['Incident No.','Description','cfs-date','Block Location','Latitude','Longitude','drone-id']

calls_for_service = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_calls-for-service-with-census-data.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)

flight_blocks = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_drone-seconds-block-count-per-flight.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)
cv_blocks = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/mapping/cv-blocks.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)
cv_blocks['GEOID20'] = cv_blocks['GEOID20'].apply(lambda x: x[3:])
cv_flights = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_cv-flight-manifest.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)


Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,HOUSING20,POP20,geometry
0,6,73,12600,1005,060730126001005,Block 1005,G5040,U,78661.0,U,S,20268,0,32.6326934,-117.0972764,0,0,"POLYGON ((-117.09848 32.636082, -117.098118 32..."
1,6,73,12600,1000,060730126001000,Block 1000,G5040,U,78661.0,U,S,41267,0,32.6327645,-117.0968738,0,0,"POLYGON ((-117.098419 32.636099, -117.098382 3..."
2,6,73,12600,1001,060730126001001,Block 1001,G5040,U,78661.0,U,S,19259,0,32.627561,-117.0948246,0,0,"POLYGON ((-117.095692 32.629235, -117.095466 3..."
3,6,73,12600,1002,060730126001002,Block 1002,G5040,U,78661.0,U,S,104107,0,32.6279724,-117.0932719,310,655,"POLYGON ((-117.095251 32.629345, -117.095177 3..."
4,6,73,12600,1003,060730126001003,Block 1003,G5040,U,78661.0,U,S,31836,0,32.6284473,-117.0915697,39,93,"POLYGON ((-117.092611 32.630074, -117.092519 3..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1954,6,73,10016,1000,060730100161000,Block 1000,G5040,R,NaN,NaN,S,1051164,0,32.5937593,-116.9514464,0,0,"POLYGON ((-116.966156 32.589925, -116.965916 3..."
1955,6,73,10016,1002,060730100161002,Block 1002,G5040,R,NaN,NaN,S,2009800,0,32.5850417,-116.9902654,0,0,"POLYGON ((-117.005217 32.592376, -117.005127 3..."
1956,6,73,13322,3005,060730133223005,Block 3005,G5040,U,78661.0,U,S,6217,0,32.6288505,-116.9924214,4,21,"POLYGON ((-116.993114 32.628806, -116.993053 3..."
1957,6,73,13322,3000,060730133223000,Block 3000,G5040,U,78661.0,U,S,310639,0,32.634625,-116.991596,162,574,"POLYGON ((-116.996325 32.633525, -116.996153 3..."


In [2]:
calls_for_service["cfs-date"] = pd.to_datetime(calls_for_service["Date"])
cv_flights["drone-date"] = pd.to_datetime(cv_flights["date"])


## Finding the Flight

Posada says:
- He was at a Circle K on Broadway in Chula Vista.
- He says he was arguing by a bus stop and took off biking near a bridge.
- He says this was a month and a half from when the interview occured in October. Checking all flights then after August 1st.

Census Geographies:
 - That circle K: 060730125023005.
 - The bridge on broadway: 060730124011019, 060730124011021, 060730124011022.
 
This code will look at all flights that spent at least a minute on these blocks. 

In [3]:
circle_k = flight_blocks[(flight_blocks['GEOID20']=='060730124011022')|(flight_blocks['GEOID20']=='060730124011021')|
(flight_blocks['GEOID20']=='060730124011019')]

circle_k = circle_k[circle_k['seconds'].astype(int)>60]
circle_k_flight_ids = circle_k['id'].drop_duplicates().values
circle_k_flights = cv_flights[cv_flights['id'].isin(circle_k_flight_ids)]
circle_k_flights = circle_k_flights[circle_k_flights["drone-date"]>"2023-08-01"]


In [4]:
circle_k_flights[['id','incident_id','address_map','type','match']]

,id,incident_id,address_map,type,match
9136,c4bb9f05df4dcda1cfeaca1caa04a4c9,L067086,00 N Fourth Ave,Disorderly Person,67086
9146,5644ea311790a5c5879992aea7b603ba,L067030,00 Fourth Ave,Disorderly Person,67030
9188,5dead61fec3d34d46f0428c7dc72ba0c,Cvl67832,45 north broadway Avenue,domestic dispute two subjects seen at the bus ...,67832
9304,ebc0251dba61bff565579155b4965cb9,L069691,500 C St,Located Stolen Vehicle,69691
9369,f471a964830d565c2f51cef1d7a7eb0e,CVL071349,500 Block Sea Vale Court,Domestic Disturbance,71349
9407,225d9c8c355d4a2c5173dc9944a0b9c4,CVL72635,20 Fourth Ave.,Suspicious Person,72635
9468,85b27bf8efa65dc254c95a0370a65db4,L073711,00 N Broadway,Domestic Violence,73711
9481,fabe3f6806aa9de2e185852aff4acb35,L073991 / L073997,00 N Broadway / 500 Park Wy,Impaired Driver / Person with Gun,73997
9546,d9bbd3c361074200ab0e6bf8b1f5b0ab,CVL75427,00 N. Broadway,Fire,75427


## Flight: 5dead61fec3d34d46f0428c7dc72ba0c

The description for the above fight is Domestic Dispute Two Subects Seen at the Bus Stop. This matches the date time and location of Posada's claims. 

- Likely Flight: https://app.airdata.com/flight/5dead61fec3d34d46f0428c7dc72ba0c
- Corresponding Police Report from CVPD: https://www.muckrock.com/foi/chula-vista-58/cvl67832-156153/#comms


In [5]:
calls_for_service[calls_for_service['Incident No.'].astype(str).str.contains('67832')][['Incident No.', 'Date', 'Description', 'Priority', 'Disposition',
       'Call Source', 'Zip Code', 'Block Location', 'Case No', 'Address','Latitude', 'Longitude']]

,Incident No.,Date,Description,Priority,Disposition,Call Source,Zip Code,Block Location,Case No,Address,Latitude,Longitude
78113,210814L00067832,2021-08-14 15:23:04.020,[Protected Call Type],3,ASSISTED,citizen,91911,1200 BLOCK BROADWAY,2110238.0,"1200 BLOCK BROADWAY, Chula Vista, CA, 91911",32.606672,-117.079998
132476,220817L00067832,2022-08-17 15:18:51.750,Petty Theft <=$950,5,REPORT TAKEN,citizen,91911,1600 BLOCK HILLTOP DR,2210811.0,"1600 BLOCK HILLTOP DR, Chula Vista, CA, 91911",32.597558,-117.051656


## Flight Path

It flew northwest at 300 feet above the ground, passing directly over a preschool and a church, then near a financial services center used by the city’s immigrant communities to send money to their families. En route to Posada, the drone — a Matrice 300 RTK — would pass through the airspace of TK blocks, potentially exposing the TK residents, many of whom are Latine, to the high resolution gaze of a law enforcement drone investigating an incident that had nothing to do with them. 



In [6]:
posada_flight = flight_blocks[flight_blocks['id']=='5dead61fec3d34d46f0428c7dc72ba0c']
logger.info(f"Flight: 5dead61fec3d34d46f0428c7dc72ba0c")
logger.info(f"Total Blocks: {posada_flight.shape[0]}")
logger.info(f"Total Population: {posada_flight['POP20'].astype(float).sum()}")

2024-05-21 13:53:27.813 | INFO     | __main__:<module>:2 - Flight: 5dead61fec3d34d46f0428c7dc72ba0c
2024-05-21 13:53:27.814 | INFO     | __main__:<module>:3 - Total Blocks: 23
2024-05-21 13:53:27.814 | INFO     | __main__:<module>:4 - Total Population: 5978.0


In [7]:

posada_flight_path = gpd.read_file(f"/Volumes/easystore/Drones/flights/kml/5dead61fec3d34d46f0428c7dc72ba0c.kml",driver='KML')
coordinates_df=[]
for index, row in posada_flight_path.iterrows():

    for point in row['geometry'].coords:
        # Append the name, latitude, and longitude to the coordinates DataFrame
        coordinates_df.append({
            'name': row['Name'],
            'latitude': point[1],
            'longitude': point[0],
            'altitude': point[2],
        })

posada_flight_csv = pd.DataFrame(coordinates_df)
posada_flight_csv.to_csv('../../data/anecdotes/posada.csv',index=False)
posada_flight_path.to_csv('../../data/anecdotes/posada-path.csv',index=False)